# Projet MLOps Pompiers

In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder

In [2]:
incidents = pd.read_csv(r"LFB_Incident_data_from_2009.csv")
mobilisation = pd.read_csv(r"LFB_Mobilisation_data_from_2009.csv")

C:\Users\w_sat\AppData\Local\Temp\ipykernel_17300\454423457.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.read_csv(r"LFB_Incident_data_from_2009.csv")
C:\Users\w_sat\AppData\Local\Temp\ipykernel_17300\454423457.py:2: DtypeWarning: Columns (0,13) have mixed types. Specify dtype option on import or set low_memory=False.
  mobilisation = pd.read_csv(r"LFB_Mobilisation_data_from_2009.csv")


## Exploration & Pre-processing

Dataset "Incidents"

In [3]:
incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592069 entries, 0 to 1592068
Data columns (total 39 columns):
 #   Column                                  Non-Null Count    Dtype  
---  ------                                  --------------    -----  
 0   IncidentNumber                          1592069 non-null  object 
 1   DateOfCall                              1592069 non-null  object 
 2   CalYear                                 1592069 non-null  int64  
 3   TimeOfCall                              1592069 non-null  object 
 4   HourOfCall                              1592069 non-null  int64  
 5   IncidentGroup                           1592069 non-null  object 
 6   StopCodeDescription                     1592069 non-null  object 
 7   SpecialServiceType                      506554 non-null   object 
 8   PropertyCategory                        1592069 non-null  object 
 9   PropertyType                            1592069 non-null  object 
 10  AddressQualifier              

In [4]:
incidents.isna().sum()

IncidentNumber                                  0
DateOfCall                                      0
CalYear                                         0
TimeOfCall                                      0
HourOfCall                                      0
IncidentGroup                                   0
StopCodeDescription                             0
SpecialServiceType                        1085515
PropertyCategory                                0
PropertyType                                    0
AddressQualifier                                0
Postcode_full                              778359
Postcode_district                               0
UPRN                                       141333
USRN                                       162871
IncGeo_BoroughCode                              0
IncGeo_BoroughName                              0
ProperCase                                      0
IncGeo_WardCode                                29
IncGeo_WardName                                29


In [5]:
incidents.duplicated().sum()

0

In [6]:
#Liste des colonnes à supprimer du dataframe incidents :
incid_cols = ['CalYear', 'HourOfCall', 'SpecialServiceType', 'AddressQualifier', 'Postcode_full',
             'UPRN', 'USRN','IncGeo_BoroughName', 'IncGeo_WardName', 'IncGeo_WardNameNew', 'ProperCase', 'Easting_m', 'Northing_m', 'Latitude',
               'Longitude', 'FRS','SecondPumpArriving_AttendanceTime',
                 'SecondPumpArriving_DeployedFromStation', 'Notional Cost (£)']

In [7]:
df_incid = incidents.drop(columns=incid_cols, axis=1)

In [8]:
df_incid.dropna(subset=['FirstPumpArriving_AttendanceTime', 'IncGeo_WardCode', 'IncidentStationGround', 'NumCalls'], inplace=True)

In [9]:
df_incid.isna().sum()

IncidentNumber                            0
DateOfCall                                0
TimeOfCall                                0
IncidentGroup                             0
StopCodeDescription                       0
PropertyCategory                          0
PropertyType                              0
Postcode_district                         0
IncGeo_BoroughCode                        0
IncGeo_WardCode                           0
Easting_rounded                           0
Northing_rounded                          0
IncidentStationGround                     0
FirstPumpArriving_AttendanceTime          0
FirstPumpArriving_DeployedFromStation    11
NumStationsWithPumpsAttending             0
NumPumpsAttending                         0
PumpCount                                 0
PumpMinutesRounded                        0
NumCalls                                  0
dtype: int64

In [10]:
# Remplacer les 11 NaN de DeployedFromStation par les valeurs respectives de IncidentStationGround :
df_incid['FirstPumpArriving_DeployedFromStation'] = df_incid['FirstPumpArriving_DeployedFromStation'].fillna(df_incid['IncidentStationGround'])
df_incid.isna().sum()

IncidentNumber                           0
DateOfCall                               0
TimeOfCall                               0
IncidentGroup                            0
StopCodeDescription                      0
PropertyCategory                         0
PropertyType                             0
Postcode_district                        0
IncGeo_BoroughCode                       0
IncGeo_WardCode                          0
Easting_rounded                          0
Northing_rounded                         0
IncidentStationGround                    0
FirstPumpArriving_AttendanceTime         0
FirstPumpArriving_DeployedFromStation    0
NumStationsWithPumpsAttending            0
NumPumpsAttending                        0
PumpCount                                0
PumpMinutesRounded                       0
NumCalls                                 0
dtype: int64

In [11]:
df_incid.shape

(1464222, 20)

In [12]:
df_incid['TimeOfCall'] = df_incid['DateOfCall'] + ' ' + df_incid['TimeOfCall']
df_incid.drop(['DateOfCall'], axis=1, inplace=True)
df_incid['TimeOfCall'] = pd.to_datetime(df_incid['TimeOfCall'], format='%d %b %Y %H:%M:%S')

In [13]:
df_incid['TimeOfCall'].head()

0   2009-01-01 00:00:37
2   2009-01-01 00:03:00
3   2009-01-01 00:04:27
4   2009-01-01 00:05:39
5   2009-01-01 00:06:03
Name: TimeOfCall, dtype: datetime64[ns]

In [14]:
df_incid.dtypes

IncidentNumber                                   object
TimeOfCall                               datetime64[ns]
IncidentGroup                                    object
StopCodeDescription                              object
PropertyCategory                                 object
PropertyType                                     object
Postcode_district                                object
IncGeo_BoroughCode                               object
IncGeo_WardCode                                  object
Easting_rounded                                   int64
Northing_rounded                                  int64
IncidentStationGround                            object
FirstPumpArriving_AttendanceTime                float64
FirstPumpArriving_DeployedFromStation            object
NumStationsWithPumpsAttending                   float64
NumPumpsAttending                               float64
PumpCount                                         int64
PumpMinutesRounded                              

## Dataset "Mobilisation"

In [15]:
mobilisation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211937 entries, 0 to 2211936
Data columns (total 22 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   IncidentNumber            object 
 1   CalYear                   int64  
 2   HourOfCall                int64  
 3   ResourceMobilisationId    int64  
 4   Resource_Code             object 
 5   PerformanceReporting      object 
 6   DateAndTimeMobilised      object 
 7   DateAndTimeMobile         object 
 8   DateAndTimeArrived        object 
 9   TurnoutTimeSeconds        float64
 10  TravelTimeSeconds         float64
 11  AttendanceTimeSeconds     int64  
 12  DateAndTimeLeft           object 
 13  DateAndTimeReturned       object 
 14  DeployedFromStation_Code  object 
 15  DeployedFromStation_Name  object 
 16  DeployedFromLocation      object 
 17  PumpOrder                 int64  
 18  PlusCode_Code             object 
 19  PlusCode_Description      object 
 20  DelayCodeId             

In [16]:
mobilisation.isna().sum()

IncidentNumber                    0
CalYear                           0
HourOfCall                        0
ResourceMobilisationId            0
Resource_Code                     0
PerformanceReporting              0
DateAndTimeMobilised              0
DateAndTimeMobile             27161
DateAndTimeArrived                0
TurnoutTimeSeconds            27289
TravelTimeSeconds             27382
AttendanceTimeSeconds             0
DateAndTimeLeft               46356
DateAndTimeReturned         1238779
DeployedFromStation_Code         27
DeployedFromStation_Name         27
DeployedFromLocation           1098
PumpOrder                         0
PlusCode_Code                     0
PlusCode_Description              0
DelayCodeId                 1664927
DelayCode_Description       1664927
dtype: int64

In [17]:
mobilisation.duplicated().sum()

0

In [18]:
df_mobil = mobilisation[mobilisation['PumpOrder']==1]

In [19]:
df_mobil = df_mobil[['IncidentNumber', 'PumpOrder', 'DelayCodeId']]

In [20]:
df_mobil['DelayCodeId'].value_counts()

DelayCodeId
12.0    249973
9.0      82033
5.0      27774
8.0      26343
7.0      14312
11.0      7446
13.0      6927
10.0      5582
3.0       2211
6.0       2162
Name: count, dtype: int64

In [21]:
# Remplacer les DelaiCodeId manquant par 1, supposant que le code 1 = pas de ralentissement.
df_mobil['DelayCodeId'] = df_mobil['DelayCodeId'].fillna(1)

In [22]:
df_mobil.isna().sum()

IncidentNumber    0
PumpOrder         0
DelayCodeId       0
dtype: int64

In [23]:
df_mobil.dtypes

IncidentNumber     object
PumpOrder           int64
DelayCodeId       float64
dtype: object

In [24]:
df_mobil['DelayCodeId'] = df_mobil['DelayCodeId'].astype('int')

In [25]:
df_mobil.shape

(1471194, 3)

## Merge des deux Dataset

In [26]:
df = pd.merge(left=df_incid, right=df_mobil, on='IncidentNumber', how='inner')

In [27]:
df.drop(columns=['Postcode_district', 'PumpCount', 'PropertyType'], axis = 1, inplace=True)

In [28]:
df['Year'] = df['TimeOfCall'].dt.year
df['DayOfWeek'] = df['TimeOfCall'].dt.dayofweek
df['Hour'] = df['TimeOfCall'].dt.hour

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452073 entries, 0 to 1452072
Data columns (total 21 columns):
 #   Column                                 Non-Null Count    Dtype         
---  ------                                 --------------    -----         
 0   IncidentNumber                         1452073 non-null  object        
 1   TimeOfCall                             1452073 non-null  datetime64[ns]
 2   IncidentGroup                          1452073 non-null  object        
 3   StopCodeDescription                    1452073 non-null  object        
 4   PropertyCategory                       1452073 non-null  object        
 5   IncGeo_BoroughCode                     1452073 non-null  object        
 6   IncGeo_WardCode                        1452073 non-null  object        
 7   Easting_rounded                        1452073 non-null  int64         
 8   Northing_rounded                       1452073 non-null  int64         
 9   IncidentStationGround              

In [30]:
# Profiling du dataframe incidents :
# profile = ProfileReport(df, title="Profiling Report")
# profile

In [31]:
num_cols = ['NumStationsWithPumpsAttending', 'NumPumpsAttending', 'NumCalls', 'DelayCodeId']
df[num_cols] = df[num_cols].astype(int)

In [32]:
string_cols = ['IncidentNumber', 'TimeOfCall', 'IncidentGroup', 'StopCodeDescription', 'PropertyCategory',
                'IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround', 'FirstPumpArriving_DeployedFromStation']

df[string_cols] = df[string_cols].astype(str)

In [33]:
label_encoder = LabelEncoder()
df_encoded = df.apply(label_encoder.fit_transform)

In [34]:
df_encoded.dtypes

IncidentNumber                           int32
TimeOfCall                               int32
IncidentGroup                            int32
StopCodeDescription                      int32
PropertyCategory                         int32
IncGeo_BoroughCode                       int32
IncGeo_WardCode                          int32
Easting_rounded                          int64
Northing_rounded                         int64
IncidentStationGround                    int32
FirstPumpArriving_AttendanceTime         int64
FirstPumpArriving_DeployedFromStation    int32
NumStationsWithPumpsAttending            int64
NumPumpsAttending                        int64
PumpMinutesRounded                       int64
NumCalls                                 int64
PumpOrder                                int64
DelayCodeId                              int64
Year                                     int64
DayOfWeek                                int64
Hour                                     int64
dtype: object

In [35]:
X = df_encoded.drop('FirstPumpArriving_AttendanceTime', axis=1)
y = df_encoded['FirstPumpArriving_AttendanceTime']

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Standardisation

In [38]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Linear Regression

In [39]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [40]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = lr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
mpe = np.mean((y_test - y_pred) / y_test) * 100

# Affichage des métriques
print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (R²): ", r2)
print("Root Mean Squared Error (RMSE): ", rmse)
print("Mean Percentage Error (MPE): ", mpe)

print("\nScore :")
print('score train :', lr.score(X_train, y_train))
print('score test :', lr.score(X_test, y_test))


Mean Squared Error (MSE):  10308.50893958234
Mean Absolute Error (MAE):  69.5796302585592
R-squared (R²):  0.4687078485984626
Root Mean Squared Error (RMSE):  101.53082753322923
Mean Percentage Error (MPE):  -inf

Score :
score train : 0.46929656027811506
score test : 0.4687078485984626


LightGBM

In [41]:
import lightgbm as lgb

model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train, y_train)

y_pred = model_lgb.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
mpe = np.mean((y_test - y_pred) / y_test) * 100

# Affichage des métriques
print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (R²): ", r2)
print("Root Mean Squared Error (RMSE): ", rmse)
print("Mean Percentage Error (MPE): ", mpe)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1921
[LightGBM] [Info] Number of data points in the train set: 1016451, number of used features: 19
[LightGBM] [Info] Start training from score 316.546778
Mean Squared Error (MSE):  7493.800906664875
Mean Absolute Error (MAE):  60.88479761159336
R-squared (R²):  0.6137756071987184
Root Mean Squared Error (RMSE):  86.56674249771025
Mean Percentage Error (MPE):  -inf


In [42]:
predictions = y_pred
y_true = y

# Générer un échantillon aléatoire pour l'affichage
indices = np.random.choice(len(predictions), size=3, replace=False)

# Afficher l'échantillon
for index in indices:
    print(f"Exemple {index + 1}:")
    print(f"Valeur prédite : {predictions[index]}")
    print(f"Valeur réelle : {y_true[index]}")
    print("-" * 20)

Exemple 15538:
Valeur prédite : 544.4293682364647
Valeur réelle : 356
--------------------
Exemple 147909:
Valeur prédite : 227.23883121086163
Valeur réelle : 163
--------------------
Exemple 423137:
Valeur prédite : 504.39157719647994
Valeur réelle : 532
--------------------


Features Importance : 

In [43]:
# Importance des fonctionnalités
feature_importances = model_lgb.feature_importances_

# Associer les importances aux noms de fonctionnalités
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))

# Trier les importances des fonctionnalités par ordre croissant
sorted_importances = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse = True)

# Afficher les importances des fonctionnalités
for feature, importance in sorted_importances:
    print(f'Feature: {feature}, Importance: {importance}')

Feature: Northing_rounded, Importance: 630
Feature: Easting_rounded, Importance: 610
Feature: FirstPumpArriving_DeployedFromStation, Importance: 371
Feature: DelayCodeId, Importance: 265
Feature: IncGeo_WardCode, Importance: 230
Feature: IncidentStationGround, Importance: 129
Feature: TimeOfCall, Importance: 126
Feature: Hour, Importance: 112
Feature: PropertyCategory, Importance: 111
Feature: NumPumpsAttending, Importance: 89
Feature: IncGeo_BoroughCode, Importance: 74
Feature: IncidentGroup, Importance: 52
Feature: NumStationsWithPumpsAttending, Importance: 43
Feature: StopCodeDescription, Importance: 41
Feature: PumpMinutesRounded, Importance: 39
Feature: NumCalls, Importance: 39
Feature: IncidentNumber, Importance: 20
Feature: DayOfWeek, Importance: 14
Feature: Year, Importance: 5
Feature: PumpOrder, Importance: 0
